In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import sys
sys.path.append('../src')
from lib.data_load.csvs import get_asset_price_history
from lib.strategies import WeeklyDCA, MonthlyDCA, WeeklyFNGDCA, WeeklyCategoricalFNGDCA, WeeklyRollingZScoreDCA, WeeklyCategoricalZScoreDCA, WeeklyZFNGDCA

# Test setup

strategies = {
    "weekly": {
        "params":["Tuesday"], 
        "strategies": [ WeeklyDCA, WeeklyFNGDCA, WeeklyCategoricalFNGDCA, WeeklyRollingZScoreDCA, WeeklyCategoricalZScoreDCA, WeeklyZFNGDCA]
    },
    "monthly": {
        "params": [16],
        "strategies": [ MonthlyDCA ]
    }
}

periods = [("Last 3 Years", 365*3), ("Last 2 Years", 365*2), ("Last year", 365), ("Last Semester", 365//2)]

# Set experiment parameters

START = "2017-01-01"
END = "2022-05-08"
ASSET = "BTC"
WEEKLY_INVESTMENT = 100

# Collect BTC data

df = get_asset_price_history(ASSET, START, END)

df = WeeklyFNGDCA(WEEKLY_INVESTMENT, "Tuesday").preprocess(df)
df = WeeklyCategoricalFNGDCA(WEEKLY_INVESTMENT, "Tuesday").preprocess(df)
df = WeeklyRollingZScoreDCA(WEEKLY_INVESTMENT, "Tuesday").preprocess(df)
df = WeeklyCategoricalZScoreDCA(WEEKLY_INVESTMENT, "Tuesday").preprocess(df)
df = MonthlyDCA(WEEKLY_INVESTMENT*4, 15).preprocess(df)

# Generate stats
res = []
for label, period in periods:
    df_period = df.iloc[-period:]
    for k, v in strategies.items():
        for param in v["params"]:
            for strategy_class in v["strategies"]:
                investment = WEEKLY_INVESTMENT * 4 if k == "monthly" else WEEKLY_INVESTMENT
                strategy = strategy_class(investment, param)
                stats = strategy.backtest(strategy.apply(df_period))
                stats["period"] = label
                res.append(stats)
                
df_res = pd.DataFrame(res)

In [18]:
def style(df):
    return df.style.highlight_max(subset=["strategy_fiat_gain_absolute", "strategy_fiat_gain_percent"], color = 'lightgreen', axis = 0).highlight_min(subset=["strategy_average_asset_price"], color = 'lightgreen', axis = 0)

for period, _ in periods:
    df_res_period = df_res[df_res.period==period]
    print(period)
    display(style(df_res_period))

Last 3 Years


,strategy,params,total_operations,total_fiat_invested,strategy_asset_position,strategy_fiat_value,strategy_fiat_gain_absolute,strategy_fiat_gain_percent,strategy_average_asset_price,period
0,Weekly DCA,"{'dca_amount': 100, 'day_of_week': 'Tuesday'}",156,15600.000000,1.061378,36149.756658,20549.756658,131.729209,14697.873316,Last 3 Years
1,Weekly FNG DCA,"{'dca_amount': 100, 'day_of_week': 'Tuesday'}",156,15656.000000,1.147980,39099.340650,23443.340650,149.740295,13637.873523,Last 3 Years
2,Weekly FNG Categorical DCA,"{'dca_amount': 100, 'day_of_week': 'Tuesday'}",156,16650.000000,1.255760,42770.258169,26120.258169,156.878427,13258.904597,Last 3 Years
3,Weekly Z-Score DCA,"{'dca_amount': 100, 'day_of_week': 'Tuesday'}",156,14340.599601,0.987127,33620.805133,19280.205532,134.444905,14527.620297,Last 3 Years
4,Weekly Categorical Z-Score DCA,"{'dca_amount': 100, 'day_of_week': 'Tuesday'}",156,17200.000000,1.146203,39038.846611,21838.846611,126.970038,15006.062412,Last 3 Years
5,Weekly Z+FNG DCA,"{'dca_amount': 100, 'day_of_week': 'Tuesday'}",156,18250.000000,1.340585,45659.348122,27409.348122,150.188209,13613.457555,Last 3 Years
6,Monthly DCA,"{'dca_amount': 400, 'day_of_month': 16}",36,14400.000000,1.008828,34359.945341,19959.945341,138.610732,14273.987346,Last 3 Years


Last 2 Years


,strategy,params,total_operations,total_fiat_invested,strategy_asset_position,strategy_fiat_value,strategy_fiat_gain_absolute,strategy_fiat_gain_percent,strategy_average_asset_price,period
7,Weekly DCA,"{'dca_amount': 100, 'day_of_week': 'Tuesday'}",104,10400.000000,0.448928,15290.173834,4890.173834,47.020902,23166.274390,Last 2 Years
8,Weekly FNG DCA,"{'dca_amount': 100, 'day_of_week': 'Tuesday'}",104,9612.000000,0.409248,13938.673384,4326.673384,45.013248,23487.002828,Last 2 Years
9,Weekly FNG Categorical DCA,"{'dca_amount': 100, 'day_of_week': 'Tuesday'}",104,9900.000000,0.426279,14518.753034,4618.753034,46.654071,23224.221041,Last 2 Years
10,Weekly Z-Score DCA,"{'dca_amount': 100, 'day_of_week': 'Tuesday'}",104,9429.272348,0.391227,13324.905316,3895.632968,41.314248,24101.791640,Last 2 Years
11,Weekly Categorical Z-Score DCA,"{'dca_amount': 100, 'day_of_week': 'Tuesday'}",104,11700.000000,0.521664,17767.493736,6067.493736,51.858921,22428.228412,Last 2 Years
12,Weekly Z+FNG DCA,"{'dca_amount': 100, 'day_of_week': 'Tuesday'}",104,11200.000000,0.499015,16996.072936,5796.072936,51.750651,22444.230291,Last 2 Years
13,Monthly DCA,"{'dca_amount': 400, 'day_of_month': 16}",24,9600.000000,0.417670,14225.516616,4625.516616,48.182465,22984.680192,Last 2 Years


Last year


,strategy,params,total_operations,total_fiat_invested,strategy_asset_position,strategy_fiat_value,strategy_fiat_gain_absolute,strategy_fiat_gain_percent,strategy_average_asset_price,period
14,Weekly DCA,"{'dca_amount': 100, 'day_of_week': 'Tuesday'}",52,5200.000000,0.119976,4086.298640,-1113.701360,-21.417334,43341.957316,Last year
15,Weekly FNG DCA,"{'dca_amount': 100, 'day_of_week': 'Tuesday'}",52,6286.000000,0.151900,5173.614939,-1112.385061,-17.696231,41382.388566,Last year
16,Weekly FNG Categorical DCA,"{'dca_amount': 100, 'day_of_week': 'Tuesday'}",52,6950.000000,0.170489,5806.745023,-1143.254977,-16.449712,40764.988845,Last year
17,Weekly Z-Score DCA,"{'dca_amount': 100, 'day_of_week': 'Tuesday'}",52,5573.606227,0.132649,4517.931031,-1055.675195,-18.940613,42017.669956,Last year
18,Weekly Categorical Z-Score DCA,"{'dca_amount': 100, 'day_of_week': 'Tuesday'}",52,4700.000000,0.102081,3476.803120,-1223.196880,-26.025466,46041.878962,Last year
19,Weekly Z+FNG DCA,"{'dca_amount': 100, 'day_of_week': 'Tuesday'}",52,6450.000000,0.152594,5197.249503,-1252.750497,-19.422488,42268.946902,Last year
20,Monthly DCA,"{'dca_amount': 400, 'day_of_month': 16}",12,4800.000000,0.108441,3693.403829,-1106.596171,-23.054087,44263.904667,Last year


Last Semester


,strategy,params,total_operations,total_fiat_invested,strategy_asset_position,strategy_fiat_value,strategy_fiat_gain_absolute,strategy_fiat_gain_percent,strategy_average_asset_price,period
21,Weekly DCA,"{'dca_amount': 100, 'day_of_week': 'Tuesday'}",26,2600.000000,0.058412,1989.454567,-610.545433,-23.482517,44511.743114,Last Semester
22,Weekly FNG DCA,"{'dca_amount': 100, 'day_of_week': 'Tuesday'}",26,3418.000000,0.078786,2683.393988,-734.606012,-21.492277,43383.331128,Last Semester
23,Weekly FNG Categorical DCA,"{'dca_amount': 100, 'day_of_week': 'Tuesday'}",26,3900.000000,0.090215,3072.669926,-827.330074,-21.213592,43229.874715,Last Semester
24,Weekly Z-Score DCA,"{'dca_amount': 100, 'day_of_week': 'Tuesday'}",26,2945.668679,0.066928,2279.503183,-666.165496,-22.615086,44012.797490,Last Semester
25,Weekly Categorical Z-Score DCA,"{'dca_amount': 100, 'day_of_week': 'Tuesday'}",26,2050.000000,0.044581,1518.410969,-531.589031,-25.931172,45983.265360,Last Semester
26,Weekly Z+FNG DCA,"{'dca_amount': 100, 'day_of_week': 'Tuesday'}",26,3350.000000,0.076385,2601.626328,-748.373672,-22.339513,43856.620993,Last Semester
27,Monthly DCA,"{'dca_amount': 400, 'day_of_month': 16}",6,2400.000000,0.053034,1806.303808,-593.696192,-24.737341,45253.869885,Last Semester
